In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report as cr
import pickle

warnings.simplefilter(action = 'ignore', category = FutureWarning)
k_folds = KFold(n_splits = 10)

In [2]:
dataset = pd.read_csv("IMDB_dataset_preprocessed.csv")
dataset.drop(dataset.tail(45000).index, inplace = True)
reviews = [w for w in dataset["review"]]

In [3]:
pickle_in = open("fasttext_vectors_preprocessed.pickle", "rb")
word_vectors_fastText = pickle.load(pickle_in)
del word_vectors_fastText[-45000:]

In [4]:
def getReviewVector(review):
    return np.mean([word for word in review], axis = 0)

In [5]:
x = []
for i in range(len(word_vectors_fastText)):
    x.append(getReviewVector(word_vectors_fastText[i]))

In [6]:
y = pd.get_dummies(dataset["sentiment"])
y = y.iloc[:,1].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [8]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [9]:
score = accuracy_score(y_test, y_pred)
print(score)
print(cr(y_test, y_pred))

0.806
              precision    recall  f1-score   support

           0       0.84      0.78      0.81       525
           1       0.77      0.83      0.80       475

    accuracy                           0.81      1000
   macro avg       0.81      0.81      0.81      1000
weighted avg       0.81      0.81      0.81      1000



In [10]:
# 10-fold cross-validation
scores = cross_val_score(rfc, x, y, cv = k_folds)

In [11]:
# 10-fold cross-validation results
print("CV scores: ", scores)
print("Avg CV score: ", scores.mean())

CV scores:  [0.806 0.82  0.796 0.806 0.79  0.802 0.804 0.806 0.82  0.814]
Avg CV score:  0.8064
